# The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2019 Semester 1
-----
## Project 2: Short Text Location Prediction
-----
###### Student Name(s): Qini ZHANG (901051)
###### Python version: Python 3
###### Submission deadline: 1pm, Friday 24 May 2019
## RAW Results Part

## Import Libray and learners

In [187]:
import math
import numpy as np
import pandas as pd
import nltk
import re
import string
%matplotlib inline
pd.set_option('display.max_colwidth', 150)
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
# from sklearn.neural_network import MLPClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [3]:
from nltk.corpus import stopwords 
from nltk import word_tokenize
from nltk.data import load
from nltk.stem import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import string
from string import digits

In [4]:
dev_raw = pd.read_csv('dev-raw.tsv', sep = '\\t', header=0)
test_raw = pd.read_csv('test-raw.tsv', sep = '\\t', header=0)
train_raw = pd.read_csv('train-raw.tsv', sep = '\\t', header=0)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel 

## Helper Functions

In [120]:
%matplotlib inline
pd.set_option('display.max_colwidth', 100)

In [4]:
def outcome(clf, X_test, y_test):
    y_pred = clf.predict(X_test)
    
    df = pd.DataFrame(confusion_matrix(y_test, y_pred, labels = geo), index = geo, columns = geo)
    
    sns.heatmap(df, annot = True, fmt = 'd')
    
    plt.title("Confusion Matrix")
    plt.xlabel("Predicted Label")
    plt.ylabel("Real Label")
    plt.yticks(rotation=0)
    plt.show()
    print("======================================================")
    print(classification_report(y_test, y_pred, target_names = geo))
    print("======================================================")
    # ACC:
    acc = accuracy_score(y_pred, y_test)
    print("Acc: {:.2%}".format(acc))
    
    return y_pred

In [6]:
def remove_website(text):
#     text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)#, flags=re.MULTILINE)
    text = re.sub(r"http\S+", "", text)
    return text
def remove_punct(text):
    text  = "".join([char.lower() for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

# remove emoji but keep some special language character:
# def remove_emoji(text):
#     emoji_pattern = re.compile("["
#         u"\U0001F600-\U0001F64F"  # emoticons
#         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
#         u"\U0001F680-\U0001F6FF"  # transport & map symbols
#         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
#         u"\U0001F1F2-\U0001F1F4"  # Macau flag
#         u"\U0001F1E6-\U0001F1FF"  # flags
#         u"\U0001F600-\U0001F64F"
#         u"\U00002702-\U000027B0"
#         u"\U000024C2-\U0001F251"
#         u"\U0001f926-\U0001f937"
#         u"\U0001F1F2"
#         u"\U0001F1F4"
#         u"\U0001F620"
#         u"\u200d"
#         u"\u2640-\u2642"
#         "]+", flags=re.UNICODE)
#     return emoji_pattern.sub('', text)

# split camel usernames
def convert_usrname(name):
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1 \2', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1 \2', s1).lower()

def remove_longtail(text):
    return re.sub(r'(.)\1{2,}',r'\1',text)

def tokenization(text):
    text = re.split('\W+', text)
    return text

def remove_unicode(text):
    return ' '.join([i for i in text.split() if '\\' not in i])
def remove_stopwords(text):
    text = [word for word in text.split() if word not in stopword]
    text = ' '.join(text)
    return text

#version2
# def remove_stopwords(text):
#     return ' '.join([wordnet.lemmatize(i) for i in text.split() if i not in stopwords])

# def remove_username(text):
# #     return ", ".join(re.findall("(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9]+)",text, re.I ))
#     text = re.sub('@[^\s]+','',text)
#     return text

In [109]:
import nltk
nltk.download('stopwords')
stopword = nltk.corpus.stopwords.words('english')
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /Users/Eva/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/Eva/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [110]:
wn = nltk.WordNetLemmatizer()

def lemmatizer(text):
    text = [wn.lemmatize(word) for word in text]
    text = ''.join(text)
    return text

In [111]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/Eva/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Perform preprocessing


using original pure tfidf with dev input describe below: 
1. camel case add, rm unicode and puct -> 34.0%(SGD)
2. just camel case add -> 34.23%(MNB)
3. camel + stop words -> 34.35%(SGD)
4. stop words -> 34.55%(SGD)
5. nothing -> 34.58%(SGD)

using fansy preprocessing and with feature selection chi2
6. camel SGD ->34.38%
7. allin -> 33.38% (MNB)

## Feature Selection
* chi2 performs beter than PMI

In [ ]:
# 80% -> 50196
# 50% -> 31373
# 30% -> 18823
# 10% -> 6274
k_best = SelectKBest(mutual_info_classif, k=50196).fit(cv, y_train)
k_best_features = [features[i] for i in k_best.get_support(indices=True)]

with open('best_features.txt', 'w') as f:
    for item in k_best_features:
        f.write("%s\n" % item)

In [ ]:
K_BEST_FEATURES = list()
with open('best_features.txt', 'r') as f:
    for line in f:
        K_BEST_FEATURES.extend(line.strip().split('\n'))
        
def feature_select(text):
    web = remove_website(text)
    unicode = remove_unicode(web)
    usrname = convert_usrname(unicode)
    punct = remove_punct(usrname)
    token = tokenization(punct)
    nonstop = remove_stopwords(token)
    finish = lemmatizer(nonstop)
    return [i for i in finish if i in K_BEST_FEATURES]

### Apply Feature selection (1) -> 80% features

In [ ]:
vectorizer = CountVectorizer(analyzer=feature_select).fit(X_train)
X_train_freq = vectorizer.transform(X_train)
X_dev_freq = vectorizer.transform(X_dev) # We fit Dev to Train
X_test_freq = vectorizer.transform(X_test) # We fit Test to Train

### Apply Feature selection (2) -> tfidf

In [ ]:
tfidf = TfidfTransformer() # transform frequency to tfidf
X_train_tfidf = tfidf.fit_transform(X_train_freq)
X_dev_tfidf = tfidf.fit_transform(X_dev_freq)
X_test_tfidf = tfidf.fit_transform(X_test_freq)

In [26]:
tv = TfidfVectorizer(sublinear_tf=True)

# tv = TfidfVectorizer(sublinear_tf=True, lowercase=True, analyzer='word', stop_words=stop, ngram_range=(1,1))

In [27]:
tfid_train = tv.fit_transform(train_raw['Text'].values)

## GridSearch 
## test tfidf & SGD parameters, with unpreprocessing datasets


In [8]:
X_train = train_raw['Text']
y_train = train_raw['Location']
X_dev = dev_raw['Text']
y_dev = dev_raw['Location']
X_test = test_raw['Text']

In [10]:
from pprint import pprint
from time import time
import logging

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', SGDClassifier(tol=None)),
])

In [ ]:
parameters = {
    'tfidf__stop_words':('english',None),
    'tfidf__max_features':(1000,5000,10000,30000,50000,None),
    'tfidf__norm':('l1','l2',None),
    'tfidf__use_idf':(True, False),
#     'clf__max_iter': (20,),
    'clf__alpha': (0.00001, 0.000001, 0.0001),
    'clf__penalty': ('l2', 'elasticnet','l1'),
    'clf__max_iter': (1000, 2000, 3000, 500),
}

In [ ]:
grid_search = GridSearchCV(pipeline,
                           parameters,
                           cv=3,
                           scoring='accuracy',
                           n_jobs=-1, 
                           verbose=1)

print("Performing grid search...")
print("pipeline:", [name for name, _ in pipeline.steps])
print("parameters:")
pprint(parameters)
t0 = time()

grid_search.fit(X_train, y_train)

print("done in %0.3fs" % (time() - t0))
print()

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))


In [258]:
dev_raw = pd.read_csv('dev-raw.tsv', sep = '\\t', header=0)
test_raw = pd.read_csv('test-raw.tsv', sep = '\\t', header=0)
train_raw = pd.read_csv('train-raw.tsv', sep = '\\t', header=0)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel 

In [5]:
print(train_raw.shape)
print(dev_raw.shape)

(103364, 3)
(37316, 3)


In [133]:
row.shape

(140680, 3)

In [269]:
row = pd.concat([train_raw,dev_raw])

## Trial for rm dev and train's http and punt and camel

In [270]:
row.head()

,Instance_ID,Location,Text,web,username
0,1,Melbourne,"""\ud83c\udf17 @ Melbourne, Victoria, Australia...","""\ud83c\udf17 @ Melbourne, Victoria, Australia","""\ud83c\udf17 @ melbourne, victoria, austra..."
1,2,Melbourne,"""@theage Of course it costs more, minimum stan...","""@theage Of course it costs more, minimum stan...","""@theage of course it costs more, minimum sta..."
2,3,Brisbane,"""Hope people make just as much noise as they d...","""Hope people make just as much noise as they d...",""" hope people make just as much noise as they ..."
3,4,Perth,"""Pouring the perfect Prosecco \ud83e\udd42\ud8...","""Pouring the perfect Prosecco \ud83e\udd42\ud8...",""" pouring the perfect prosecco \ud83e\udd42\u..."
4,5,Perth,"""$LNY losing traction at 0.014, see this retra...","""$LNY losing traction at 0.014, see this retra...","""$lny losing traction at 0.014, see this retra..."


In [270]:
punct = '["\'?#@*_\-,\.]' 
abbr_dict={
    punct: ' ',
    "sydney":" sydney ",
    "melbourne":" melbourne ",
    "brisbane":" brisbane ",
    "queensland":" brisbane ",
    "perth":" perth ",
    
    
    "western australia":" perth ",
    "westernaustralia":" perth ",
    "new south wales":" sydney ",
    "newsouthwales":" sydney ",
    "gold coast":" brisbane ",
    "goldcoast":" brisbane ",
    "whitsunday":" brisbane ",
    
    " syd ":" sydney ",
    " vic ":" melbourne ",
    " mel ":" melbourne ",
    " nsw ":" sydney ",
    " qld ":" brisbane ",
    " bne ":" brisbane ",
    " wa ":" perth ",
    
    "SYD":" sydney ",
    "VIC":" melbourne ",
    "MEL":" melbourne ",
    "NSW":" sydney ",
    "QLD":" brisbane ",
    "BNE":" brisbane ",
    "WA":" perth ",
    
#     " syd":" sydney ",
#     " vic":" melbourne ",
#     " nsw":" sydney ",
#     " qld":" brisbane ",
    '\s+':' ',
}

In [260]:
train_raw['web'] = train_raw['Text'].apply(lambda x: remove_website(x))
# for NSW
train_raw['abbr'] = train_raw['web'].replace(abbr_dict, regex=True)

train_raw['username'] = train_raw['abbr'].apply(lambda x: convert_usrname(x))
train_raw['abbr'] = train_raw['username'].replace(abbr_dict, regex=True)

# train_raw['longtail'] = train_raw['abbr'].apply(lambda x: remove_longtail(x))

# version2:

# train_raw['tokenized'] = train_raw['longtail'].apply(lambda x: tokenization(x))

# train_raw['nonstop'] = train_raw['longtail'].apply(lambda x: remove_stopwords(x))

# train_raw['finish'] = train_raw['nonstop'].apply(lambda x: lemmatizer(x)) 
# with numbers
# train_raw['finish'] = train_raw['username'].apply(lambda x: remove_punct(x))

In [261]:
# dev
# dev_raw['web'] = dev_raw['Text'].apply(lambda x: remove_website(x))
# dev_raw['username'] = dev_raw['web'].apply(lambda x: convert_usrname(x))
# dev_raw['finish'] = dev_raw['username'].apply(lambda x: remove_longtail(x))

dev_raw['web'] = dev_raw['Text'].apply(lambda x: remove_website(x))
# NSW
dev_raw['abbr'] = dev_raw['web'].replace(abbr_dict, regex=True)
dev_raw['username'] = dev_raw['abbr'].apply(lambda x: convert_usrname(x))
dev_raw['abbr'] = dev_raw['username'].replace(abbr_dict, regex=True)
# dev_raw['longtail'] = dev_raw['abbr'].apply(lambda x: remove_longtail(x))

# version2:

# dev_raw['tokenized'] = dev_raw['longtail'].apply(lambda x: tokenization(x))

# dev_raw['nonstop'] = dev_raw['longtail'].apply(lambda x: remove_stopwords(x))

# dev_raw['finish'] = dev_raw['nonstop'].apply(lambda x: lemmatizer(x)) 

In [222]:
row = pd.concat([train_raw,dev_raw])
dev_raw.head(100)

,Instance_ID,Location,Text,web,abbr,username
0,21,Brisbane,You's suit,You's suit,you s suit,you s suit
1,22,Perth,@KellyFrye @girlscouts when life gives you cookies... \ud83c\udf6a\ud83c\udf6a\ud83c\udf6a\ud83c\udf6a,@KellyFrye @girlscouts when life gives you cookies... \ud83c\udf6a\ud83c\udf6a\ud83c\udf6a\ud83c\udf6a,kelly frye girlscouts when life gives you cookies \ud83c\udf6a\ud83c\udf6a\ud83c\udf6a\ud83c\udf6a,kelly frye girlscouts when life gives you cookies \ud83c\udf6a\ud83c\udf6a\ud83c\udf6a\ud83c\udf6a
2,23,Brisbane,@noviarezki naaaah itu dia hahaha,@noviarezki naaaah itu dia hahaha,noviarezki naaaah itu dia hahaha,noviarezki naaaah itu dia hahaha
3,24,Melbourne,#LouisWhyAreYouAnEGG bro whyyy??? http://t.co/i4AhvJHPbj,#LouisWhyAreYouAnEGG bro whyyy???,louis why are you an egg bro whyyy,louis why are you an egg bro whyyy
4,25,Brisbane,@9NewsBrisbane Why do we have to put up with all this advertising on the screen????????????,@9NewsBrisbane Why do we have to put up with all this advertising on the screen????????????,9 news brisbane why do we have to put up with all this advertising on the screen,9 news brisbane why do we have to put up with all this advertising on the screen
5,26,Melbourne,"""\Twt: 20 creepy kids' drawings that will haunt your nightmares... http://t.co/4WUa8JHAdy http://t.co/CwXSFRtuDL\""""""""""","""\Twt: 20 creepy kids' drawings that will haunt your nightmares...",\ twt: 20 creepy kids drawings that will haunt your nightmares,\ twt: 20 creepy kids drawings that will haunt your nightmares
6,27,Perth,"""Temp: 21.5\u00b0C. Wind:11.5km/h. Pressure: 1019.7 hPa, Falling slowly. Humidity 55%. Rain Today 0.0mm. #HammondPark #Weather""","""Temp: 21.5\u00b0C. Wind:11.5km/h. Pressure: 1019.7 hPa, Falling slowly. Humidity 55%. Rain Today 0.0mm. #HammondPark #Weather""",temp: 21 5\u00b0 c wind:11 5km/h pressure: 1019 7 h pa falling slowly humidity 55% rain today 0 0mm hammond park weather,temp: 21 5\u00b0 c wind:11 5km/h pressure: 1019 7 h pa falling slowly humidity 55% rain today 0 0mm hammond park weather
7,28,Melbourne,In situ... http://t.co/X96m46OuOL,In situ...,in situ,in situ
8,29,Melbourne,An out-of-sessions court hearing will be held very shortly in relation to stabbing murder of 17 year old Masa Vukotic.@TheTodayShow,An out-of-sessions court hearing will be held very shortly in relation to stabbing murder of 17 year old Masa Vukotic.@TheTodayShow,an out of sessions court hearing will be held very shortly in relation to stabbing murder of 17 year old masa vukotic the today show,an out of sessions court hearing will be held very shortly in relation to stabbing murder of 17 year old masa vukotic the today show
9,210,Perth,"""Saye laper! Betul betul laper! (with anandita, dcsendy, and 2 others at Makan 2 Cafe) \u2014 https://t.co/FaG0tQzYYI""","""Saye laper! Betul betul laper! (with anandita, dcsendy, and 2 others at Makan 2 Cafe) \u2014",saye laper! betul betul laper! (with anandita dcsendy and 2 others at makan 2 cafe) \u2014,saye laper! betul betul laper! (with anandita dcsendy and 2 others at makan 2 cafe) \u2014


In [262]:
# test
test_raw['web'] = test_raw['Text'].apply(lambda x: remove_website(x))
# NSW
test_raw['abbr'] = test_raw['web'].replace(abbr_dict, regex=True)
test_raw['username'] = test_raw['abbr'].apply(lambda x: convert_usrname(x))
test_raw['abbr'] = test_raw['username'].replace(abbr_dict, regex=True)
# test_raw['longtail'] = test_raw['abbr'].apply(lambda x: remove_longtail(x))

# # version2:
# test_raw['tokenized'] = test_raw['longtail'].apply(lambda x: tokenization(x))

# test_raw['nonstop'] = test_raw['longtail'].apply(lambda x: remove_stopwords(x))

# test_raw['finish'] = test_raw['nonstop'].apply(lambda x: lemmatizer(x)) 

# with numbers
# test_raw['finish'] = test_raw['username'].apply(lambda x: remove_punct(x))

In [224]:
test_raw.head(20)

,Instance_ID,Location,Text,web,abbr,username
0,31,?,"""Awesome @evomagazine cover, time to dive in yet another issue. https://t.co/pecFlufAYT""","""Awesome @evomagazine cover, time to dive in yet another issue.",awesome evomagazine cover time to dive in yet another issue,awesome evomagazine cover time to dive in yet another issue
1,32,?,@InceptCruze goes 4-30 smfh,@InceptCruze goes 4-30 smfh,incept cruze goes 4 30 smfh,incept cruze goes 4 30 smfh
2,33,?,Sad loss to business &amp; sailing worlds. #RIP #sydneytohobart #sydhob https://t.co/viy0npcWwW,Sad loss to business &amp; sailing worlds. #RIP #sydneytohobart #sydhob,sad loss to business &amp; sailing worlds rip sydney tohobart sydhob,sad loss to business &amp; sailing worlds rip sydney tohobart sydhob
3,34,?,@heavenlogin I would like my pokemon back in pokemon go I had a mew and ots gone like everything else pls give me back my pokemon Heaven Pls,@heavenlogin I would like my pokemon back in pokemon go I had a mew and ots gone like everything else pls give me back my pokemon Heaven Pls,heavenlogin i would like my pokemon back in pokemon go i had a mew and ots gone like everything else pls give me back my pokemon heaven pls,heavenlogin i would like my pokemon back in pokemon go i had a mew and ots gone like everything else pls give me back my pokemon heaven pls
4,35,?,#Hit30 Girls Talk Boys 5SOS,#Hit30 Girls Talk Boys 5SOS,hit30 girls talk boys 5 sos,hit30 girls talk boys 5 sos
5,36,?,*X-ellent https://t.co/oDtqUD8H1i,*X-ellent,x ellent,x ellent
6,37,?,Dean Braun. Take a bow. How good are your runners presented tonight. Stunning,Dean Braun. Take a bow. How good are your runners presented tonight. Stunning,dean braun take a bow how good are your runners presented tonight stunning,dean braun take a bow how good are your runners presented tonight stunning
7,38,?,***sigh*** Not sure which coverage is going to be more nauseating. Fox or Channel 9 for #NRLBulldogsDragons ?,***sigh*** Not sure which coverage is going to be more nauseating. Fox or Channel 9 for #NRLBulldogsDragons ?,sigh not sure which coverage is going to be more nauseating fox or channel 9 for nrl bulldogs dragons,sigh not sure which coverage is going to be more nauseating fox or channel 9 for nrl bulldogs dragons
8,39,?,public dm'yi kapattim kapatali cirkin kizlardan gereksiz iltifatlar almaz oldum,public dm'yi kapattim kapatali cirkin kizlardan gereksiz iltifatlar almaz oldum,public dm yi kapattim kapatali cirkin kizlardan gereksiz iltifatlar almaz oldum,public dm yi kapattim kapatali cirkin kizlardan gereksiz iltifatlar almaz oldum
9,310,?,@fullcammunism it's an American smoker/chargriller and I can't imagine my life without it.,@fullcammunism it's an American smoker/chargriller and I can't imagine my life without it.,fullcammunism it s an american smoker/chargriller and i can t imagine my life without it,fullcammunism it s an american smoker/chargriller and i can t imagine my life without it


In [235]:
test_raw.iloc[[59455]]

,Instance_ID,Location,Text,web,abbr,username
59455,359456,?,View from Port Macquarie Lighthouse #travel #roadtrip #NewSouthWales https://t.co/GqJr0eTx5T,View from Port Macquarie Lighthouse #travel #roadtrip #NewSouthWales,view from port macquarie lighthouse travel roadtrip sydney,view from port macquarie lighthouse travel roadtrip new south wales


## RF

In [20]:
clf2 = RandomForestClassifier().fit(tfid_train, train_raw["Location"].values)

In [21]:
# train
y_pred_RF = clf2.predict(tfid_train)
y_train_RF = train_raw["Location"].values
acc = accuracy_score(y_pred_RF, y_train_RF)
print("Acc: {:.2%}".format(acc))
# print((clf2.predict(tfid_train) == train_raw["Location"]).mean())

Acc: 97.55%
0.9755233930575442


In [22]:
# dev
dfid_test = tv.transform(dev_raw['username'].values)
dy_pred_RF = clf2.predict(dfid_test)
dy_test_RF = dev_raw["Location"]
acc = accuracy_score(dy_pred_RF, dy_test_RF)
print("Acc: {:.2%}".format(acc))
# print((rcf.predict(dfid) == dev_top10["Location"]).mean())
print((clf2.predict(dfid_test) == dev_raw["Location"]).mean())
# print((pd.Series(rcf.predict(dfid), dev_top10.index).apply(lambda x: groups.get(x, '?')) == dev_top10["Location"]).mean())

Acc: 32.82%
0.32822381820130775


## MNB

In [32]:
clf

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [23]:
clf = MultinomialNB()

In [36]:
clf.fit(tfid_train, train_raw["Location"].values)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [37]:
y_pred_MNB = clf.predict(tfid_train)
y_train_MNB = train_raw["Location"].values
acc = accuracy_score(y_pred_MNB, y_train_MNB)
print("Acc: {:.2%}".format(acc))
print((clf.predict(tfid_train) == train_raw["Location"]).mean())

Acc: 75.12%
0.7511802948802291


In [38]:
# dev
dfid_test = tv.transform(dev_raw['username'].values)
dy_pred_MNB = clf.predict(dfid_test)
dy_test_MNB = dev_raw["Location"]
acc = accuracy_score(dy_pred_MNB, dy_test_MNB)
print("Acc: {:.2%}".format(acc))
# print((rcf.predict(dfid) == dev_top10["Location"]).mean())
print((clf.predict(dfid_test) == dev_raw["Location"]).mean())
# print((pd.Series(rcf.predict(dfid), dev_top10.index).apply(lambda x: groups.get(x, '?')) == dev_top10["Location"]).mean())

Acc: 34.94%
0.34939436166791726


## SGD
### treat training dataset as testing set 
* do nothing -> 87.15%

In [263]:
# tv = TfidfVectorizer(sublinear_tf=True)
tv = TfidfVectorizer()
# tv = TfidfVectorizer(sublinear_tf=True, lowercase=True, analyzer='word', stop_words='english', ngram_range=(1,1))
tv

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [264]:
# rclf = SGDClassifier(random_state=41, max_iter=1000, tol=None, n_jobs=-1)
rclf = SGDClassifier(random_state=41, max_iter=1000, tol=None, n_jobs=-1, alpha=1e-05, penalty='l2')
# rclf = SGDClassifier(n_jobs=-1)
rclf

SGDClassifier(alpha=1e-05, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=1000, n_iter=None,
       n_jobs=-1, penalty='l2', power_t=0.5, random_state=41, shuffle=True,
       tol=None, verbose=0, warm_start=False)

### dev 
* 34.60% without stopwords
* add stopwords -> 34.55
* rm special character -> 34.17% => hell no
* rm camel, http for all -> 35.35% rd state:41
        * only web -> 34.79
        * add longtail -> 35.44
        * web usmane stopwords lemmmizer -> 35.22
        * perform stopwords in tfidf -> 35.31
        * cm, web, punt, unicode -> 34.68
        * all in -> 34.61
        * change alpha to defalut -> 35.18
                        * add eng -> 35.41% -> 36.08 kaggle
        * change max_iter = 2000 istead of 1000 -> 35.34%
                        * add stop_words = eng
        * add abbr  -> 35.33% max_iter = 1000
                        -> 35.35% 2000
                    *rm punct max 1000 -> 35.09
                    * max 2000 -> 35.10
        * tfidf() -> 35.21
             

## crazy row - combine dev and train

In [265]:
#86.27 88.79
tfid_train_row = tv.fit_transform(row['abbr'].values)

rclf.fit(tfid_train_row, row["Location"].values)

# # y_pred_row = rclf.predict(tfid_train_row)
# y_train_row = row["Location"].values
# acc = accuracy_score(y_pred_row, y_train_row)
# print("Acc: {:.2%}".format(acc))
# print((rclf.predict(tfid_train_row) == row["Location"]).mean())

SGDClassifier(alpha=1e-05, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=1000, n_iter=None,
       n_jobs=-1, penalty='l2', power_t=0.5, random_state=41, shuffle=True,
       tol=None, verbose=0, warm_start=False)

## Output

In [266]:
X_test = tv.transform(test_raw['abbr'].values)
y_test_pred = rclf.predict(X_test)

In [267]:
output = test_raw[['Instance_ID']]
output['Class'] = y_test_pred
output.shape

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


(108148, 2)

In [268]:
output.columns = ['Id', 'Class']
output.head(20)

,Id,Class
0,31,Perth
1,32,Perth
2,33,Sydney
3,34,Sydney
4,35,Perth
5,36,Perth
6,37,Sydney
7,38,Sydney
8,39,Sydney
9,310,Sydney


In [269]:
output.to_csv('9.csv', index=False)